In [1]:
# install mish-cuda if you want to use mish activation
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install
%cd ..

Cloning into 'mish-cuda'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 195 (delta 20), reused 20 (delta 20), pack-reused 169 (from 1)
Receiving objects: 100% (195/195), 203.57 KiB | 1.02 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/mish-cuda
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/extension.py:139: UserWarning: Unknown Extension options: 'headers'
  warnings.warn(msg)
running build
running build_py
creating build/lib.linux-x86_64-cpython-310/mish_cuda
copying src/mish_cuda/__init__.py -> build/lib.linux-x86_64-cpython-310/mish_cuda
running egg_info
creating src/mish_cuda.egg-info
writing src/mish_cuda.egg-info/PKG-INFO
writing dependency_links to src/mish_cuda.egg-info/dependency_links.txt
writing requirements to src/mish_cuda.egg-info/requires.txt
writing top-level names to src/mish_cuda.egg-info/top_level.txt
writing manifest file 'src/mish_cuda

Restart the session after installing mish-cuda : Runtime > Restart session

In [1]:
!pip install pyngrok

In [2]:
!unzip grocery_detect_submission.zip

Archive:  grocery_detect_submission.zip
  inflating: C1_P02_N1_S5_1.JPG      
  inflating: C1_P02_N2_S2_1.JPG      
  inflating: C1_P03_N2_S2_1.JPG      
  inflating: C1_P06_N1_S3_1.JPG      
  inflating: C1_P11_N2_S4_3.JPG      
  inflating: grocery_detect_client.ipynb  
  inflating: grocery_detection_server.ipynb  
  inflating: last_yolov4-p5-results_strip.pt  
 extracting: models/__init__.py      
  inflating: models/__pycache__/__init__.cpython-37.pyc  
  inflating: models/__pycache__/common.cpython-37.pyc  
  inflating: models/__pycache__/experimental.cpython-37.pyc  
  inflating: models/__pycache__/yolo.cpython-37.pyc  
  inflating: models/common.py        
  inflating: models/experimental.py  
  inflating: models/export.py        
  inflating: models/yolo.py          
  inflating: models/yolov4-csp.yaml  
  inflating: models/yolov4-p5.yaml   
  inflating: models/yolov4-p6.yaml   
  inflating: models/yolov4-p7.yaml   
 extracting: utils/__init__.py       
  inflating: utils/__pyc

In [3]:
import torch
import torch.nn as nn
import cv2
import numpy as np
import random
from pathlib import Path
import base64
from flask import Flask, request, jsonify, send_file
import logging

from utils.datasets import letterbox
from utils.general import (
    check_img_size, non_max_suppression, plot_one_box, scale_coords
)

In [4]:
# Initialize the Flask application
app = Flask(__name__)

# Configure logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Set device for computation (CPU or GPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [5]:
# Load YOLO model
def load_model():
    """Load the YOLOv4 model from the specified file."""
    logging.info("Loading the YOLO model.")
    model = torch.load('/content/last_yolov4-p5-results_strip.pt', map_location=device)['model'].float().fuse().eval()
    logging.info("Model loaded successfully.")
    return model

# Perform object detection
def detection(img0):
    """Perform object detection on the given image.

    Args:
        img0 (numpy.ndarray): The input image for detection.

    Returns:
        numpy.ndarray: The processed image with detected objects plotted.
    """
    logging.info("Starting detection process.")
    imgsz = 896
    imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size

    # Resize and preprocess the image
    img = letterbox(img0, new_shape=imgsz)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img)
    img = img.to(device, non_blocking=True)
    img = img.float()
    img /= 255.0  # Normalize to 0.0 - 1.0

    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    pred = model(img, augment=False)[0]

    conf_thres = 0.4
    iou_thres = 0.6
    pred = non_max_suppression(pred, conf_thres=conf_thres, iou_thres=iou_thres, merge=False)

    # Get class names and colors
    names = model.module.names if hasattr(model, 'module') else model.names
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    for i, det in enumerate(pred):  # detections per image
        gn = torch.tensor(img0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
        if det is not None and len(det):
            # Rescale boxes to original image size
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()

            # Print results
            for c in det[:, -1].unique():
                n = (det[:, -1] == c).sum()  # detections per class
                logging.info(f"{n} {names[int(c)]} detected.")

            # Draw results on the image
            for *xyxy, conf, cls in det:
                label = f'{names[int(cls)]}'
                plot_one_box(xyxy, img0, label=label, color=colors[int(cls)], line_thickness=2)

    logging.info("Detection process completed.")
    return img0

In [6]:
# Define the detection endpoint
@app.route('/detect', methods=['POST'])
def detect():
    """API endpoint to handle image upload and perform object detection."""
    if 'image' not in request.files:
        logging.error("No image file provided in the request.")
        return jsonify({'error': 'No image provided'}), 400

    file = request.files['image']
    img0 = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)

    if img0 is None:
        logging.error("Invalid image file.")
        return jsonify({'error': 'Invalid image'}), 400

    pred = detection(img0)

    # Encode the processed image to Base64
    _, buffer = cv2.imencode('.jpg', pred)
    image_base64 = base64.b64encode(buffer).decode('utf-8')

    logging.info("Image processed successfully and encoded to Base64.")
    return jsonify({'image': image_base64})

# Define a health check endpoint
@app.get('/health')
def health():
    """API endpoint to check service health status."""
    logging.info("Health check endpoint accessed.")
    return jsonify({'message': 'OK'}), 200

In [ ]:
from pyngrok import ngrok
from google.colab import userdata
auth_token = userdata.get('auth_token') #add auth_token secrets of ngrok

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000)

# Print the public URL
print('Public URL:', ngrok_tunnel.public_url)

# Run the server
model = load_model()
app.run(host='0.0.0.0', port=8000)

Public URL: https://147f-34-125-4-238.ngrok-free.app


<ipython-input-5-4cd4a2795cdf>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/content/last_yolov4-p5-results_strip.pt', map_location=device)['model'].

Fusing layers... Model Summary: 331 layers, 7.02988e+07 parameters, 6.81919e+07 gradients
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 15:00:07] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 15:02:01] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 15:02:31] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 15:02:56] "POST /detect HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 15:03:51] "POST /detect HTTP/1.1" 200 -
INF